# 📊 WhatsApp Wrapped

**Create beautiful Spotify Wrapped-style visualizations for your WhatsApp group chats!**

This notebook analyzes your WhatsApp chat exports and generates stunning HTML reports with:
- 📈 Rich Analytics (message counts, activity patterns, emoji usage)
- 🎨 Beautiful Visualizations (interactive Plotly charts with dark theme)
- 👥 User Insights (top contributors, activity sparklines)
- 📅 Calendar Heatmaps (activity across the year)
- 💬 Message Analysis (word patterns, response times)

---

## 🚀 How to Use

1. **Run the Setup cell** below to install dependencies
2. **Configure your options** using the interactive form
3. **Upload your chat export** (.zip or .txt file)
4. **Click Generate** and download your report!

### 📱 How to Export WhatsApp Chat
1. Open WhatsApp and navigate to the group chat
2. Tap the group name → More → Export chat
3. Choose **"Without Media"** for faster processing
4. Save the `.zip` file to upload here

---

🔒 **Privacy First**: All processing happens in this notebook. Your data is never uploaded anywhere else.

In [ ]:
#@title 🔧 **Step 1: Setup** (Run this first!) { display-mode: "form" }
#@markdown This cell installs all required dependencies and clones the WhatsApp Wrapped repository.
#@markdown 
#@markdown **Click the play button ▶️ to run this cell.**

import subprocess
import sys
from IPython.display import display, HTML, clear_output

print("📦 Installing dependencies...")
print("="*50)

# Install required packages (suppress most output)
packages = [
    "pandas>=2.0.0",
    "numpy>=1.24.0",
    "matplotlib>=3.7.0",
    "seaborn>=0.12.0",
    "plotly>=5.14.0",
    "plotly-calplot>=0.1.20",
    "wordcloud>=1.9.0",
    "pillow>=10.0.0",
    "python-dateutil>=2.8.0",
    "pyyaml>=6.0",
    "tqdm>=4.65.0",
    "jinja2>=3.1.0",
    "emojis>=0.7.0",
]

for pkg in packages:
    pkg_name = pkg.split(">=")[0].split("[")[0]
    print(f"  • Installing {pkg_name}...", end=" ")
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", pkg],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print("✓")
    else:
        print("⚠️ (may already be installed)")

print("\n📥 Cloning WhatsApp Wrapped repository...")
print("="*50)

import os
repo_path = "/content/whatsapp-wrapped"

if os.path.exists(repo_path):
    print("  • Repository already exists, updating...")
    result = subprocess.run(
        ["git", "-C", repo_path, "pull", "--quiet"],
        capture_output=True, text=True
    )
    print("  ✓ Repository updated!")
else:
    result = subprocess.run(
        ["git", "clone", "--depth", "1", 
         "https://github.com/fer-lion/whatsapp-wrapped.git", repo_path],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print("  ✓ Repository cloned successfully!")
    else:
        print(f"  ⚠️ Clone failed: {result.stderr}")

# Add to Python path
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print("\n" + "="*50)
print("✅ Setup complete! Proceed to Step 2.")
print("="*50)

# Display styled success message
display(HTML("""
<div style="background: linear-gradient(135deg, #1DB954, #191414); 
            padding: 20px; border-radius: 10px; margin-top: 20px;
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
    <h3 style="color: white; margin: 0;">🎉 Ready to go!</h3>
    <p style="color: #b3b3b3; margin: 10px 0 0 0;">All dependencies installed. Run the next cell to configure and generate your report.</p>
</div>
"""))

In [ ]:
#@title 🎛️ **Step 2: Configure & Generate Report** { display-mode: "form" }
#@markdown ### Configuration Options
#@markdown Adjust these settings before uploading your chat file.

#@markdown ---

#@markdown **Year Filter** - Analyze only messages from a specific year
year_filter = "All Years" #@param ["All Years", "2024", "2023", "2022", "2021", "2020", "2019", "2018", "2017", "2016", "2015"]

#@markdown **Minimum Messages** - Exclude users with fewer messages than this
min_messages = 2 #@param {type:"slider", min:1, max:50, step:1}

#@markdown **Fixed Layout** - Force desktop layout (disable responsive scaling)
fixed_layout = False #@param {type:"boolean"}

#@markdown ---
#@markdown ### Upload & Generate
#@markdown After setting options above, **run this cell** and upload your WhatsApp export.

from google.colab import files
from IPython.display import display, HTML, clear_output
from datetime import datetime
from pathlib import Path
import tempfile
import os

# Display configuration summary
display(HTML(f"""
<div style="background: #282828; padding: 15px; border-radius: 8px; 
            font-family: 'Courier New', monospace; margin-bottom: 20px;">
    <h4 style="color: #1DB954; margin: 0 0 10px 0;">📋 Current Configuration</h4>
    <table style="color: #b3b3b3; border-collapse: collapse;">
        <tr><td style="padding: 5px 20px 5px 0;">Year Filter:</td>
            <td style="color: white;">{year_filter}</td></tr>
        <tr><td style="padding: 5px 20px 5px 0;">Min Messages:</td>
            <td style="color: white;">{min_messages}</td></tr>
        <tr><td style="padding: 5px 20px 5px 0;">Fixed Layout:</td>
            <td style="color: white;">{'Yes' if fixed_layout else 'No'}</td></tr>
    </table>
</div>
"""))

print("📤 Please upload your WhatsApp chat export (.zip or .txt)...")
print("="*60)

try:
    uploaded = files.upload()
except Exception as e:
    print(f"\n⚠️ Upload cancelled or failed: {e}")
    uploaded = None

if uploaded:
    # Get the uploaded filename
    filename = list(uploaded.keys())[0]
    file_content = uploaded[filename]
    
    print(f"\n✓ Uploaded: {filename} ({len(file_content):,} bytes)")
    print("\n" + "="*60)
    print("🔄 Generating your WhatsApp Wrapped report...")
    print("="*60 + "\n")
    
    # Save the uploaded file temporarily
    temp_dir = tempfile.mkdtemp()
    chat_path = os.path.join(temp_dir, filename)
    with open(chat_path, 'wb') as f:
        f.write(file_content)
    
    try:
        # Import the required modules
        from src.parser import parse_whatsapp_export
        from src.analytics import analyze_chat, format_hour, get_hour_emoji
        from src.charts import ChartCollection, create_user_sparkline, create_user_hourly_sparkline, chart_to_html
        from jinja2 import Environment, FileSystemLoader
        
        # Parse year filter
        year_value = None if year_filter == "All Years" else int(year_filter)
        
        # Step 1: Parse the chat
        print("[1/4] 📖 Parsing chat file...")
        df, metadata = parse_whatsapp_export(
            chat_path,
            filter_system=True,
            min_messages=min_messages,
            year_filter=year_value,
        )
        print(f"      ✓ Found {len(df):,} messages from {metadata.total_members} members")
        
        # Step 2: Run analytics
        print("[2/4] 📊 Running analytics...")
        analytics = analyze_chat(df)
        print(f"      ✓ Analyzed {analytics.total_days} days of chat history")
        
        # Step 3: Generate charts
        print("[3/4] 📈 Generating visualizations...")
        chart_collection = ChartCollection(analytics)
        charts_html = chart_collection.to_html_dict(include_plotlyjs_first=True)
        
        # Generate user sparklines
        user_sparklines = {}
        user_hourly_sparklines = {}
        for user_stat in analytics.user_stats[:12]:
            sparkline_fig = create_user_sparkline(user_stat.daily_activity, user_stat.name)
            user_sparklines[user_stat.name] = chart_to_html(sparkline_fig, include_plotlyjs=False)
            hourly_sparkline_fig = create_user_hourly_sparkline(user_stat.hourly_activity, user_stat.name)
            user_hourly_sparklines[user_stat.name] = chart_to_html(hourly_sparkline_fig, include_plotlyjs=False)
        print(f"      ✓ Created {len(charts_html)} charts and {len(user_sparklines)} sparklines")
        
        # Step 4: Render HTML report
        print("[4/4] 🎨 Rendering HTML report...")
        template_dir = Path("/content/whatsapp-wrapped/src/templates")
        env = Environment(
            loader=FileSystemLoader(template_dir),
            autoescape=False,
        )
        template = env.get_template("report.html")
        
        formatted_hour = format_hour(analytics.most_active_hour)
        hour_emoji = get_hour_emoji(analytics.most_active_hour)
        
        html_content = template.render(
            metadata=metadata,
            analytics=analytics,
            charts=charts_html,
            user_sparklines=user_sparklines,
            user_hourly_sparklines=user_hourly_sparklines,
            generation_date=datetime.now().strftime("%Y-%m-%d %H:%M"),
            fixed_layout=fixed_layout,
            formatted_hour=formatted_hour,
            hour_emoji=hour_emoji,
        )
        print("      ✓ Report rendered successfully!")
        
        # Save the report
        stem = Path(filename).stem.replace(" ", "_")
        output_filename = f"{stem}_report.html"
        output_path = os.path.join(temp_dir, output_filename)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print("\n" + "="*60)
        print("🎉 REPORT GENERATED SUCCESSFULLY!")
        print("="*60)
        
        # Display summary stats
        display(HTML(f"""
        <div style="background: linear-gradient(135deg, #1DB954, #191414); 
                    padding: 25px; border-radius: 12px; margin: 20px 0;
                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
            <h2 style="color: white; margin: 0 0 15px 0;">📊 {metadata.filename}</h2>
            <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px;">
                <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                    <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_messages:,}</div>
                    <div style="color: #b3b3b3; font-size: 12px;">MESSAGES</div>
                </div>
                <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                    <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_members}</div>
                    <div style="color: #b3b3b3; font-size: 12px;">MEMBERS</div>
                </div>
                <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 8px; text-align: center;">
                    <div style="color: #1DB954; font-size: 28px; font-weight: bold;">{analytics.total_days}</div>
                    <div style="color: #b3b3b3; font-size: 12px;">DAYS</div>
                </div>
            </div>
            <div style="margin-top: 15px; padding-top: 15px; border-top: 1px solid rgba(255,255,255,0.1);">
                <span style="color: #b3b3b3;">Date Range:</span>
                <span style="color: white;">{metadata.date_range_start.strftime('%b %d, %Y')} - {metadata.date_range_end.strftime('%b %d, %Y')}</span>
            </div>
        </div>
        """))
        
        # Download button
        print("\n📥 Downloading your report...")
        files.download(output_path)
        
        display(HTML("""
        <div style="background: #282828; padding: 20px; border-radius: 10px; margin-top: 20px;
                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
            <h4 style="color: #1DB954; margin: 0 0 10px 0;">💡 What's Next?</h4>
            <ul style="color: #b3b3b3; margin: 0; padding-left: 20px;">
                <li>Open the downloaded HTML file in any web browser</li>
                <li>Share the report with your group members</li>
                <li>Print to PDF from your browser for a permanent copy</li>
            </ul>
        </div>
        """))
        
    except Exception as e:
        import traceback
        print(f"\n❌ Error generating report: {e}")
        print("\nFull error details:")
        traceback.print_exc()
        
        display(HTML("""
        <div style="background: #5c1c1c; padding: 20px; border-radius: 10px; margin-top: 20px;">
            <h4 style="color: #ff6b6b; margin: 0 0 10px 0;">⚠️ Troubleshooting Tips</h4>
            <ul style="color: #ffb3b3; margin: 0; padding-left: 20px;">
                <li>Make sure you uploaded a valid WhatsApp export file (.zip or .txt)</li>
                <li>The file should be exported "Without Media" from WhatsApp</li>
                <li>Try running the Setup cell again if you see import errors</li>
                <li>Check that your chat has enough messages for the selected year filter</li>
            </ul>
        </div>
        """))
    
    finally:
        # Cleanup temp files
        import shutil
        try:
            shutil.rmtree(temp_dir)
        except:
            pass
else:
    display(HTML("""
    <div style="background: #3d3d00; padding: 20px; border-radius: 10px; margin-top: 20px;">
        <h4 style="color: #ffeb3b; margin: 0 0 10px 0;">📤 No file uploaded</h4>
        <p style="color: #fff9c4; margin: 0;">Run this cell again and upload your WhatsApp chat export file.</p>
    </div>
    """))

---

## 📚 Additional Information

### Supported Export Formats
- **iOS**: `.zip` file containing `_chat.txt`
- **Android**: `.zip` or `.txt` file

### What's Analyzed?
| Metric | Description |
|--------|-------------|
| Message counts | Total messages per user |
| Activity patterns | Hourly and daily trends |
| Emoji usage | Top emojis per user and overall |
| Calendar heatmap | Activity visualization by day |
| Response patterns | Conversation dynamics |

### Privacy
- All processing happens locally in this Colab notebook
- Your chat data is never stored or uploaded to any server
- The generated report is saved only to your browser's downloads

---

**Made with ❤️ for WhatsApp users who love data**

[GitHub Repository](https://github.com/fer-lion/whatsapp-wrapped) | [Report Issues](https://github.com/fer-lion/whatsapp-wrapped/issues)